# Simulating mating

Mating regimes (`xftsim.mate.MatingRegime`) serve two primary functions:

 1. the mates per female the number and offspring count / sex per mating
 2. deciding who mates with whom

 
## Mating and offspring counts

The first (the mates per female the number and offspring count / sex per mating) is determined by the following arguments to the `MatingRegime` constructor:

 - `mates_per_female` - the number of mating partners per female
 - `offspring_per_pair` - the number of offspring per mate pair
 - `female_offspring_per_pair` - the number of female offspring per mate pair
 
Each of this can be constant or variable.

### Variable counts

`xftsim.utils` provides the `VariableCount` class for random counts which include a `draw` method for generating random variates,  an `expectation` property that returns the expected value, and a `nonzero_fraction` property that returns the expected number of nonzero counts. The later is useful for determining, e.g., the fraction of couples expected to produce offspring. 

The simplest `VariableCount` subclass is the `ConstantCount`:

In [1]:
import xftsim as xft

ccount = xft.utils.ConstantCount(2)
ccount.draw(3), ccount.expectation, ccount.nonzero_fraction

(array([2, 2, 2]), 2, 1.0)

Other useful `VariableCount` subclasses are demonstrated below:

In [2]:
pcount = xft.utils.PoissonCount(2)
nbcount = xft.utils.NegativeBinomialCount(2,.5)
ztpcount = xft.utils.ZeroTruncatedPoissonCount(2)
mixcount = xft.utils.MixtureCount(componentCounts=[xft.utils.ConstantCount(0),
                                                   xft.utils.ConstantCount(3)],
                                  mixture_probabilities= [.4,.6])

for count in [pcount, nbcount, ztpcount, mixcount]:
    print(count.expectation, count.nonzero_fraction)

2 0.8646647167633873
2.0 0.75
2.3130352854993315 1
1.7999999999999998 0.6


 
## Who with whom?

The second (who mates with whom) is mostly determined by the `.mate()` method of the `MatingRegime` but also by two boolean flags provided to the `MatingRegime` constructor:

 - `sex_aware` if `True`, enforces a regime where males may only mate with females and vice versa. In many cases, we have no interest in sex effects and it is therefore convenient to set this to `False` to increase the effective population size.
 
 - `exhaustive` if `False` when assigning mates, male mates are sampled with replacement. This too is convenient to set to `False` but should have little impact either way in sufficiently large simulations.
 
The `mate()` methods are specific each `MatingRegime` subclass. Generally speaking, they map haplotype and phenotype arrays to a `MateAssignment` object. 

In what follows, we first introduce simple random mating regimes, then  `MateAssignment` objects, and finally we'll move on to more complex mating regimes.

Throughout, we'll reference the toy dataset below:

In [3]:
import xftsim as xft

demo = xft.sim.DemoSimulation('BGRM')
demo

<DemoSimulation>
Bivariate GCTA with balanced random mating demo

n = 2000; m = 400
Two phenotypes, height and bone mineral denisty (BMD)
assuming bivariate GCTA infinitessimal archtecture
with h2 values set to 0.5 and 0.4 for height and BMD
respectively and a genetic effect correlation of 0.0.

## Random mating

The simplest mating regime is that of random mating.

In [4]:
from xftsim.mate import RandomMatingRegime
from xftsim.utils import ConstantCount, PoissonCount

rm_regime = RandomMatingRegime(offspring_per_pair = ConstantCount(2),
                                mates_per_female = ConstantCount(1),
                                female_offspring_per_pair = 'balanced',
                                sex_aware = False,
                                exhaustive = True)
                                        

Setting `female_offspring_per_pair` to `"balanced"` rather than a `VariableCount` object will result in a equal number of male and female offspring. 

It is sometimes useful to predict how the population size will change across generations. The `MatingRegime.population_growth_factor` property reveals the expected multiplicative change in population size each generation.

As we specified `rm_regime` such that each female is paired with exactly one male and each mating will produce exactly one offspring, we expect constant population size:

In [5]:
rm_regime.population_growth_factor

1.0

Of course, this is not always the case, as we demonstrate below:

In [6]:
rm_regime2 = RandomMatingRegime(offspring_per_pair = PoissonCount(2.2),
                                mates_per_female = PoissonCount(1.1),
                                female_offspring_per_pair = 'balanced',
                                sex_aware = False,
                                exhaustive = True)
              
rm_regime2.population_growth_factor

1.2100000000000002

## Mate assignments

Mating regime objects map haplotypes and phenotypes to a `MateAssignment` object, which stores information about who mates with whom and the offspring such matings produce:

:::{note}

In practice, users will rarely call `MateAssignment.mate()` directly as this occurs automatically when running a simulation.

:::


In [7]:
mate_assignment = rm_regime.mate(haplotypes=demo.haplotypes,
                                 phenotypes=demo.phenotypes)

:::{note}

Since the mating regime we're using is random, the only information the `mate` method uses from either the haplotypes or phenotypes would be the associated `index.SampleIndex` object.

:::

Two useful perspectives for viewing a `MateAssignment` are the `mating_frame`, which is indexed by couple, and the `reproduction_frame`, which is indexed by offspring:

In [8]:
mate_assignment.get_mating_frame()

,maternal_sample,maternal_iid,maternal_fid,maternal_sex,paternal_sample,paternal_iid,paternal_fid,paternal_sex,n_offspring,n_female_spring
0,0..1_1980.1_990,1_1980,1_990,1,0..1_321.1_160,1_321,1_160,1,2,0
1,0..1_782.1_391,1_782,1_391,0,0..1_1766.1_883,1_1766,1_883,0,2,1
2,0..1_1899.1_949,1_1899,1_949,1,0..1_772.1_386,1_772,1_386,0,2,2
3,0..1_229.1_114,1_229,1_114,1,0..1_1129.1_564,1_1129,1_564,1,2,1
4,0..1_1260.1_630,1_1260,1_630,1,0..1_1745.1_872,1_1745,1_872,1,2,1
...,...,...,...,...,...,...,...,...,...,...
995,0..1_912.1_456,1_912,1_456,0,0..1_430.1_215,1_430,1_215,1,2,2
996,0..1_1892.1_946,1_1892,1_946,1,0..1_838.1_419,1_838,1_419,1,2,0
997,0..1_642.1_321,1_642,1_321,0,0..1_260.1_130,1_260,1_130,0,2,2
998,0..1_75.1_37,1_75,1_37,1,0..1_1063.1_531,1_1063,1_531,1,2,1


In [9]:
mate_assignment.get_reproduction_frame()

,iid,fid,sex,maternal_sample,maternal_iid,maternal_fid,maternal_sex,paternal_sample,paternal_iid,paternal_fid,paternal_sex
sample,,,,,,,,,,,
0..2_0.2_0,2_0,2_0,1,0..1_1980.1_990,1_1980,1_990,1,0..1_321.1_160,1_321,1_160,1
0..2_1.2_0,2_1,2_0,1,0..1_1980.1_990,1_1980,1_990,1,0..1_321.1_160,1_321,1_160,1
0..2_2.2_1,2_2,2_1,0,0..1_782.1_391,1_782,1_391,0,0..1_1766.1_883,1_1766,1_883,0
0..2_3.2_1,2_3,2_1,1,0..1_782.1_391,1_782,1_391,0,0..1_1766.1_883,1_1766,1_883,0
0..2_4.2_2,2_4,2_2,0,0..1_1899.1_949,1_1899,1_949,1,0..1_772.1_386,1_772,1_386,0
...,...,...,...,...,...,...,...,...,...,...,...
0..2_1995.2_997,2_1995,2_997,0,0..1_642.1_321,1_642,1_321,0,0..1_260.1_130,1_260,1_130,0
0..2_1996.2_998,2_1996,2_998,0,0..1_75.1_37,1_75,1_37,1,0..1_1063.1_531,1_1063,1_531,1
0..2_1997.2_998,2_1997,2_998,1,0..1_75.1_37,1_75,1_37,1,0..1_1063.1_531,1_1063,1_531,1


## Nonrandom mating regimes


:::{warning}

It will be challenging to to understand this information if you haven't read [the tutorial on indexing](./indexing.ipynb)! Specifically, understanding `xftsim.index.ComponentIndex` is essential.

:::

As `xftsim` mating regimes are privy to haplotypes and phenotypes, the can be very general. Here we present a few commonly used regimes.

### Linear assortative mating

By "linear assortative mating", we refer to mating mediated through a linear combination of phenotypic components. An exchangeable linear regime is provided by `xft.mate.LinearAssortativeMatingRegime`. Concretely, assume we want an exchangeable regime for $k$ phenotypes with cross-mate correlations fixed at `r`. That is, given female and male phenotypes $Y,\tilde{Y}\in R^{n\times k}$, respectively, this regime will generate a mating permutation $P$

such that the empirical correlation is
$$\text{corr}(Y,P\tilde{Y})\approx r\cdot1_{k\times k}.$$


### Univariate primary phenotypic assortment

We can implement univariate assortative mating mediated through a single phenotype via `LinearAssortativeMatingRegime`. Below we do this for height, setting the cross-mate correlation to 0.5. We iterate for a single generation and then observe the sample mating statstics:

In [10]:
from xftsim.mate import LinearAssortativeMatingRegime

reg_pp = LinearAssortativeMatingRegime(r=.5,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name='height', 
                                                          component_name='phenotype'))

demo_pp = xft.sim.DemoSimulation()
demo_pp.mating_regime = reg_pp
demo_pp.run(1)
xmate_corr = demo_pp.results['mating_statistics']['mate_correlations']
xmate_corr.iloc[:xmate_corr.shape[0]//2,xmate_corr.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,0.268342,-0.008584,0.285039,-0.048256,0.397498,-0.042764
BMD.additiveGenetic.mother,0.008731,-0.046509,-0.004701,0.030176,0.002762,-0.003530
height.additiveNoise.mother,0.213312,0.011146,0.245511,0.047010,0.329757,0.043285
BMD.additiveNoise.mother,0.025006,0.073521,0.004836,-0.003132,0.021229,0.040452
height.phenotype.mother,0.345829,0.001849,0.380946,-0.000849,0.522178,0.000415
BMD.phenotype.mother,0.024631,0.029059,0.000926,0.015699,0.018117,0.029240


### Univariate social / genetic homogamy

Social or genetic homogamy can be modeled by specifying that mating operates only on the non-heritable or heritable components respectively:


In [11]:
from xftsim.mate import LinearAssortativeMatingRegime

reg_social = LinearAssortativeMatingRegime(r=.5,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name='height', 
                                                          component_name='additiveNoise'))

reg_genetic = LinearAssortativeMatingRegime(r=.5,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name='height', 
                                                          component_name='additiveGenetic'))


demo_soc = xft.sim.DemoSimulation(); demo_gen = xft.sim.DemoSimulation()
demo_soc.mating_regime = reg_social; demo_gen.mating_regime = reg_genetic; 
demo_soc.run(1); demo_gen.run(1)
xmate_corr_soc = demo_soc.results['mating_statistics']['mate_correlations']
xmate_corr_soc.iloc[:xmate_corr_soc.shape[0]//2,xmate_corr_soc.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,-0.075997,0.020729,-0.022070,-0.009579,-0.068682,0.006065
BMD.additiveGenetic.mother,-0.034681,0.001651,-0.049497,-0.010576,-0.058171,-0.006805
height.additiveNoise.mother,0.048020,0.039968,0.497350,-0.017413,0.372348,0.012479
BMD.additiveNoise.mother,0.024209,-0.048029,-0.028814,0.026596,-0.002513,-0.010783
height.phenotype.mother,-0.020428,0.043707,0.341454,-0.019438,0.217930,0.013352
BMD.phenotype.mother,-0.003607,-0.034945,-0.052617,0.013294,-0.038352,-0.012345


In [12]:
xmate_corr_gen = demo_gen.results['mating_statistics']['mate_correlations']
xmate_corr_gen.iloc[:xmate_corr_gen.shape[0]//2,xmate_corr_gen.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,0.529864,0.001163,0.063315,0.035746,0.415366,0.028550
BMD.additiveGenetic.mother,-0.004869,-0.007057,-0.030289,0.034901,-0.024318,0.022398
height.additiveNoise.mother,-0.027741,-0.006710,0.006791,0.031207,-0.014773,0.019760
BMD.additiveNoise.mother,-0.004571,-0.025001,0.045424,0.011315,0.028142,-0.007923
height.phenotype.mother,0.365798,-0.003683,0.050528,0.046995,0.291459,0.034050
BMD.phenotype.mother,-0.006635,-0.024104,0.016955,0.030649,0.007047,0.007698


### Exchangeable bivariate phenotypic assortment

Exchangeable bivariate phenotypic assortment is also easily accommodated in this framework. Here we implement such a regime for $r$=.1 across height and bone mineral density (BMD):

In [13]:
reg_biv = LinearAssortativeMatingRegime(r=.1,
    component_index=xft.index.ComponentIndex.from_product(phenotype_name=['height','BMD'], 
                                                          component_name='phenotype'))

demo_biv = xft.sim.DemoSimulation()
demo_biv.mating_regime = reg_biv
demo_biv.run(1)
xmate_corr = demo_biv.results['mating_statistics']['mate_correlations']
xmate_corr.iloc[:xmate_corr.shape[0]//2,xmate_corr.shape[0]//2:]

component,height.additiveGenetic.father,BMD.additiveGenetic.father,height.additiveNoise.father,BMD.additiveNoise.father,height.phenotype.father,BMD.phenotype.father
component,,,,,,
height.additiveGenetic.mother,0.042121,0.007638,0.063715,0.064258,0.076388,0.054666
BMD.additiveGenetic.mother,0.011518,-0.000970,0.025269,0.016718,0.026438,0.012362
height.additiveNoise.mother,-0.005662,0.005028,0.117724,0.087100,0.079050,0.070748
BMD.additiveNoise.mother,0.023836,0.056490,0.033903,0.063639,0.041705,0.084921
height.phenotype.mother,0.025256,0.008798,0.126190,0.105230,0.108031,0.087188
BMD.phenotype.mother,0.026047,0.044510,0.042460,0.060993,0.049401,0.075330


## Generalized assortive mating

:::{note}

Coming soon

:::

## Batched mating regimes


:::{note}

Coming soon

:::
